In [20]:
import sklearn.datasets
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch

from featurizer import *
import sklearn.model_selection
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
from sklearn.metrics import r2_score, accuracy_score

from tabdl import *

import openml
from ucimlrepo import fetch_ucirepo 

In [36]:
miami_housing = openml.datasets.get_dataset(43093)
X, y, _, _ = miami_housing.get_data(target=miami_housing.default_target_attribute, dataset_format="dataframe")

is_continuous = X.apply(lambda col: pd.api.types.is_float_dtype(col) and len(col.unique()) > 20)
X_cat = X.loc[:, ~is_continuous]
cats = list(X_cat.columns)
cat_mappings = {}
for c in cats:
    unique_values = sorted(X_cat[c].unique())
    mapping= {val: i for i, val in enumerate(unique_values)}
    X[c] = X_cat[c].map(mapping)
    cat_mappings[c] = {str(val): i for i, val in enumerate(unique_values)}

In [41]:
from featurizer import *

In [42]:
f = RegFeaturizer()
X_f = f.fit_transform(X, y)

In [45]:
X_f.isna().sum()

LATITUDE_leaf_bit0         0
LATITUDE_leaf_bit1         0
LONGITUDE_leaf_bit0        0
LONGITUDE_leaf_bit1        0
PARCELNO_leaf_bit0         0
PARCELNO_leaf_bit1         0
LND_SQFOOT_leaf_bit0       0
LND_SQFOOT_leaf_bit1       0
TOT_LVG_AREA_leaf_bit0     0
TOT_LVG_AREA_leaf_bit1     0
SPEC_FEAT_VAL_leaf_bit0    0
SPEC_FEAT_VAL_leaf_bit1    0
RAIL_DIST_leaf_bit0        0
RAIL_DIST_leaf_bit1        0
OCEAN_DIST_leaf_bit0       0
OCEAN_DIST_leaf_bit1       0
WATER_DIST_leaf_bit0       0
WATER_DIST_leaf_bit1       0
CNTR_DIST_leaf_bit0        0
CNTR_DIST_leaf_bit1        0
SUBCNTR_DI_leaf_bit0       0
SUBCNTR_DI_leaf_bit1       0
HWY_DIST_leaf_bit0         0
HWY_DIST_leaf_bit1         0
age_bit0                   0
age_bit1                   0
age_bit2                   0
age_bit3                   0
age_bit4                   0
age_bit5                   0
age_bit6                   0
avno60plus_bit0            0
month_sold_bit0            0
month_sold_bit1            0
month_sold_bit

In [27]:
is_continuous = X.apply(lambda col: pd.api.types.is_float_dtype(col) and len(col.unique()) > 20)

# Split the DataFrame based on the determined column types
X_cont = X.loc[:, is_continuous]
X_cat = X.loc[:, ~is_continuous]

In [ ]:
h

In [33]:
pd.api.types.is_float_dtype(30003)

False

In [29]:
X_cat

,age,avno60plus,month_sold,structure_quality
0,67,0,8,4
1,63,0,9,4
2,61,0,2,4
3,63,0,9,4
4,42,0,7,4
...,...,...,...,...
13927,16,0,4,4
13928,26,0,5,4
13929,16,0,7,4
13930,16,0,8,4


In [23]:
f = RegFeaturizer(bit=False)
X_t = f.fit_transform(X, y)

In [24]:
f.feature_types, f.sizes

({'LATITUDE': 'continuous',
  'LONGITUDE': 'continuous',
  'PARCELNO': 'continuous',
  'LND_SQFOOT': 'continuous',
  'TOT_LVG_AREA': 'continuous',
  'SPEC_FEAT_VAL': 'continuous',
  'RAIL_DIST': 'continuous',
  'OCEAN_DIST': 'continuous',
  'WATER_DIST': 'continuous',
  'CNTR_DIST': 'continuous',
  'SUBCNTR_DI': 'continuous',
  'HWY_DIST': 'continuous',
  'age': 'categorical',
  'avno60plus': 'categorical',
  'month_sold': 'categorical',
  'structure_quality': 'categorical'},
 {'LATITUDE': 4,
  'LONGITUDE': 4,
  'PARCELNO': 4,
  'LND_SQFOOT': 4,
  'TOT_LVG_AREA': 4,
  'SPEC_FEAT_VAL': 4,
  'RAIL_DIST': 4,
  'OCEAN_DIST': 4,
  'WATER_DIST': 4,
  'CNTR_DIST': 4,
  'SUBCNTR_DI': 4,
  'HWY_DIST': 4,
  'age': 96,
  'avno60plus': 2,
  'month_sold': 12,
  'structure_quality': 5})

In [19]:
X_t

,LATITUDE_2,LATITUDE_3,LATITUDE_5,LATITUDE_6,LONGITUDE_2,LONGITUDE_3,LONGITUDE_5,LONGITUDE_6,PARCELNO_2,PARCELNO_3,...,month_sold_8,month_sold_9,month_sold_10,month_sold_11,month_sold_12,structure_quality_1,structure_quality_2,structure_quality_3,structure_quality_4,structure_quality_5
0,0,0,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13927,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
13928,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
13929,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
13930,0,0,1,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [7]:
# dataset = sklearn.datasets.load_iris(as_frame = True)
# X: np.ndarray = dataset["data"]
# Y: np.ndarray = dataset["target"]
# task_type = 'multiclass'
# f = ClassFeaturizer(depth=3, bit=False)
# X = f.fit_transform(X, Y)
# mlp = TabDLM('MLP', task_type, n_classes=3, verbose = False)
# resnet = TabDLM('ResNet', task_type, n_classes=3, verbose = False)
# ftt = TabDLM('FTTransformer', task_type, n_classes=3, verbose = False)
# mlp.fit(X, Y)
# resnet.fit(X, Y)
# ftt.fit(X, Y)
# accuracy_score(np.argmax(mlp.predict(X), axis = 1), Y), accuracy_score(np.argmax(resnet.predict(X), axis = 1), Y), accuracy_score(np.argmax(ftt.predict(X), axis = 1), Y)

In [8]:
num_samples = 1000

# Generate continuous features
continuous_features = np.random.randn(num_samples, 3)

# Generate categorical features (numerical)
categorical_feature_1 = np.random.randint(0, 3, num_samples)  # 3 categories
categorical_feature_2 = np.random.randint(0, 5, num_samples)  # 5 categories

# Combine the features into a DataFrame
data = pd.DataFrame(continuous_features, columns=['cont_feature_1', 'cont_feature_2', 'cont_feature_3'])
data['cat_feature_1'] = categorical_feature_1
data['cat_feature_2'] = categorical_feature_2

# Generate the target variable Y as a linear combination of features + noise
coefficients = np.array([1.5, -2.0, 3.0, 0.5, -1.0])
features = np.hstack([continuous_features, categorical_feature_1.reshape(-1, 1), categorical_feature_2.reshape(-1, 1)])
noise = np.random.randn(num_samples) * 0.1  # Add some noise

Y = features.dot(coefficients) + noise
data['Y'] = Y

X = data.drop(columns = ['Y'])
Y = data['Y']

f = RegFeaturizer(depth=3, bit=False)
task_type = 'regression'
X = f.fit_transform(X, Y)
mlp = TabDLM('MLP', task_type, n_classes=3, verbose = False)
resnet = TabDLM('ResNet', task_type, n_classes=3, verbose = False)
ftt = TabDLM('FTTransformer', task_type, n_classes=3, verbose = False)
mlp.fit(X, Y)
resnet.fit(X, Y)
ftt.fit(X, Y)
r2_score(mlp.predict(X), Y), r2_score(resnet.predict(X), Y), r2_score(ftt.predict(X), Y)

(0.922578008794543, 0.8535349461659845, 0.9480954807008086)

In [9]:
dataset = sklearn.datasets.fetch_california_housing(as_frame = True)
X: np.ndarray = dataset["data"]
Y: np.ndarray = dataset["target"]
task_type = 'regression'
f = RegFeaturizer(depth=3, bit=False)
#X = f.fit_transform(X, Y)
mlp = TabDLM('MLP', task_type, verbose = False)
resnet = TabDLM('ResNet', task_type, verbose = False)
ftt = TabDLM('FTTransformer', task_type, verbose = False)
mlp.fit(X, Y)
resnet.fit(X, Y)
ftt.fit(X, Y)
r2_score(mlp.predict(X), Y), r2_score(resnet.predict(X), Y), r2_score(ftt.predict(X), Y)

(0.7169827478534101, 0.7451484736846061, 0.7885448488136712)

In [10]:
dataset = sklearn.datasets.fetch_california_housing(as_frame = True)
X: np.ndarray = dataset["data"]
Y: np.ndarray = dataset["target"]

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size =0.2, random_state = 0)

In [13]:
Y_val

14740    1.369
10101    2.413
20566    2.007
2670     0.725
15709    4.600
         ...  
6655     1.695
3505     2.046
1919     1.286
1450     2.595
4148     1.676
Name: MedHouseVal, Length: 4128, dtype: float64

In [14]:
t = np.arange(5)
type(pd.Series(t))

pandas.core.series.Series